In [1]:
import pybullet as p
import pybullet_data

In [2]:
p.connect(p.GUI)
#p.connect(p.GUI, options="--mp4=\"test4.mp4\" --mp4fps=240")
p.configureDebugVisualizer(p.COV_ENABLE_SINGLE_STEP_RENDERING,1)
p.startStateLogging(p.STATE_LOGGING_VIDEO_MP4,"./ball_r8.mp4")


plane_id = p.loadURDF(pybullet_data.getDataPath() + "/plane.urdf")
sphere_id = p.loadURDF(pybullet_data.getDataPath() + "/sphere2.urdf")

In [3]:
p.resetBasePositionAndOrientation(sphere_id,[0,0,5],[0,0,0,1.0])
p.setGravity(0,0,-9.8)

lateralFriction = 1
spinningFriction = 0
rollingFriction = 1
restitution = .8

p.changeDynamics(plane_id, -1, lateralFriction=1)
p.changeDynamics(sphere_id, -1, lateralFriction=lateralFriction)
p.changeDynamics(sphere_id, -1, spinningFriction=spinningFriction)
p.changeDynamics(sphere_id, -1, rollingFriction=rollingFriction)

p.changeDynamics(plane_id, -1, restitution=restitution)
p.changeDynamics(sphere_id, -1, restitution=restitution)

In [4]:
for i in range(500):
    p.stepSimulation()
    p.configureDebugVisualizer(p.COV_ENABLE_SINGLE_STEP_RENDERING,1)
p.disconnect()

In [4]:
p.setRealTimeSimulation(1)

In [5]:
p.getDynamicsInfo(sphere_id,-1)

(10.0,
 0.5,
 (1.0, 1.0, 1.0),
 (0.0, 0.0, 0.0),
 (0.0, 0.0, 0.0, 1.0),
 0.0,
 0.03,
 0.03,
 -1.0,
 -1.0,
 2,
 0.5,
 0.03999999910593033,
 0.03999999910593033,
 0.0,
 1e+30,
 0)

In [5]:
p.disconnect()